<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Intro" data-toc-modified-id="Intro-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Intro</a></span></li><li><span><a href="#Import-libraries-&amp;-functions,-and-connect-mongoDB" data-toc-modified-id="Import-libraries-&amp;-functions,-and-connect-mongoDB-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Import libraries &amp; functions, and connect mongoDB</a></span></li><li><span><a href="#Create-a-collection-with-offices" data-toc-modified-id="Create-a-collection-with-offices-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Create a collection with offices</a></span></li><li><span><a href="#Show-SanFran-offices-in-map" data-toc-modified-id="Show-SanFran-offices-in-map-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Show SanFran offices in map</a></span></li><li><span><a href="#Dowload-data-from-foursquare-API" data-toc-modified-id="Dowload-data-from-foursquare-API-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Dowload data from foursquare API</a></span></li><li><span><a href="#Filter-alternative-offices" data-toc-modified-id="Filter-alternative-offices-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Filter alternative offices</a></span></li><li><span><a href="#Conclusion" data-toc-modified-id="Conclusion-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Conclusion</a></span></li></ul></div>

## Intro

The objective of the project is to look for a location to set up the HQs of a new gaming company. The company will have 87 employees. We have a database with 18_800 companies (10_800 offices) with their locations that we can use as starting locations.
Different groups of employees have different requests they want for the office location. We will use the foursquare API to look for locations of coffeshops, schools, recreational places, etc. and decide the optimal location for the new office.
As a start point, I believe the San Francisco Bay Area is a good place to start looking, with a great number of international companies and local businesses and a vibrant cultural life.

## Import libraries & functions, and connect mongoDB

In [1]:
import pymongo
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np

import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster

from haversine import haversine

In [2]:
import sys

sys.path.append('../src')

from geo_functions import *

In [3]:
from pymongo import MongoClient
client = MongoClient()

In [4]:
client.ironhack

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ironhack')

In [5]:
companies = client.ironhack.companies

## Create a collection with offices

We first need to create a collection with the offices in the 'Companies' db so we can easily browse through them

In [6]:
# create the collection

offices = client.ironhack.offices

In [7]:
# populate the collection, selcting companies who don't have null values for coordinates and omiting
# old IDs so new IDs can de created that are unique for each office

res = companies.aggregate([
    {"$unwind":"$offices"},
    {"$match":{"offices.latitude":{"$ne":None}, "offices.longitude":{"$ne":None}}},
    {"$project":{"_id":0}}])

offices.insert_many(res)

Now we will make the format geoJSON

In [8]:
res2 = offices.find({},{"offices":1})

In [9]:
for comp in res2:
    geojson = {
        "type":"Point",
        "coordinates":[comp["offices"]["longitude"], comp["offices"]["latitude"]]
    } 
    offices.update_one(comp, {"$set":{"geojson":geojson}})  # We update all of the elements with the new value.

Next, we convert coordinates to an index

In [10]:
offices.create_index([("geojson", "2dsphere")])

'geojson_2dsphere'

## Show SanFran offices in map

We now look at the offices in our database an show them in a map so we know what parts of San Francisco will be more interesting for us to look at

In [11]:
# importing data from Mongo into a pandas df

world_offices_data = pd.DataFrame(list(client.ironhack.offices.find()))

In [12]:
world_offices_data.head()

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,milestones,video_embeds,screenshots,external_links,partners,geojson,deadpooled_month,deadpooled_day,deadpooled_url,ipo
0,6021ad8f292a7a01b909ef82,Wetpaint,abc2,http://www.crunchbase.com/company/wetpaint,http://wetpaint-inc.com,http://digitalquarters.net/,http://digitalquarters.net/feed/,BachelrWetpaint,web,47.0,...,"[{'id': 5869, 'description': 'Wetpaint named i...",[],"[{'available_sizes': [[[150, 86], 'assets/imag...",[{'external_url': 'http://www.geekwire.com/201...,[],"{'type': 'Point', 'coordinates': [-122.333253,...",NaN,NaN,NaN,NaN
1,6021ad8f292a7a01b909ef83,Wetpaint,abc2,http://www.crunchbase.com/company/wetpaint,http://wetpaint-inc.com,http://digitalquarters.net/,http://digitalquarters.net/feed/,BachelrWetpaint,web,47.0,...,"[{'id': 5869, 'description': 'Wetpaint named i...",[],"[{'available_sizes': [[[150, 86], 'assets/imag...",[{'external_url': 'http://www.geekwire.com/201...,[],"{'type': 'Point', 'coordinates': [-73.9964312,...",NaN,NaN,NaN,NaN
2,6021ad8f292a7a01b909ef84,AdventNet,abc3,http://www.crunchbase.com/company/adventnet,http://adventnet.com,,,manageengine,enterprise,600.0,...,[],[],"[{'available_sizes': [[[150, 94], 'assets/imag...",[],[],"{'type': 'Point', 'coordinates': [-121.904945,...",NaN,NaN,NaN,NaN
3,6021ad8f292a7a01b909ef85,Zoho,abc4,http://www.crunchbase.com/company/zoho,http://zoho.com,http://blogs.zoho.com/,http://blogs.zoho.com/feed,zoho,software,1600.0,...,"[{'id': 388, 'description': 'Zoho Reaches 2 Mi...","[{'embed_code': '<object width=""430"" height=""2...",[],[{'external_url': 'http://www.online-tech-tips...,[],"{'type': 'Point', 'coordinates': [-121.904945,...",NaN,NaN,NaN,NaN
4,6021ad8f292a7a01b909ef86,Digg,digg,http://www.crunchbase.com/company/digg,http://www.digg.com,http://blog.digg.com/,http://blog.digg.com/?feed=rss2,digg,news,60.0,...,"[{'id': 9588, 'description': 'Another Digg Exe...","[{'embed_code': '<embed src=""http://blip.tv/pl...","[{'available_sizes': [[[117, 150], 'assets/ima...",[{'external_url': 'http://www.sociableblog.com...,[],"{'type': 'Point', 'coordinates': [-122.394523,...",NaN,NaN,None,None


We will now drop all columns except for name and geojson

In [13]:
new_w_offices_data = world_offices_data[["name", "geojson"]]

In [14]:
new_w_offices_data.head()

,name,geojson
0,Wetpaint,"{'type': 'Point', 'coordinates': [-122.333253,..."
1,Wetpaint,"{'type': 'Point', 'coordinates': [-73.9964312,..."
2,AdventNet,"{'type': 'Point', 'coordinates': [-121.904945,..."
3,Zoho,"{'type': 'Point', 'coordinates': [-121.904945,..."
4,Digg,"{'type': 'Point', 'coordinates': [-122.394523,..."


We will now add columns with lat and long

In [15]:
# separate dicitonary elements in the geojson column 

geoj_df = new_w_offices_data.geojson.apply(pd.Series)

In [16]:
geoj_df.head()

,type,coordinates
0,Point,"[-122.333253, 47.603122]"
1,Point,"[-73.9964312, 40.7237306]"
2,Point,"[-121.904945, 37.692934]"
3,Point,"[-121.904945, 37.692934]"
4,Point,"[-122.394523, 37.764726]"


In [17]:
# unpack the coordinates list

new_geoj_df = pd.DataFrame(geoj_df["coordinates"].to_list(), columns=['long', 'lat'])

In [18]:
new_geoj_df.head()

,long,lat
0,-122.333253,47.603122
1,-73.996431,40.723731
2,-121.904945,37.692934
3,-121.904945,37.692934
4,-122.394523,37.764726


In [19]:
world_heatmap_df = pd.merge(new_w_offices_data, new_geoj_df, left_index=True, right_index=True)

In [20]:
world_heatmap_df

,name,geojson,long,lat
0,Wetpaint,"{'type': 'Point', 'coordinates': [-122.333253,...",-122.333253,47.603122
1,Wetpaint,"{'type': 'Point', 'coordinates': [-73.9964312,...",-73.996431,40.723731
2,AdventNet,"{'type': 'Point', 'coordinates': [-121.904945,...",-121.904945,37.692934
3,Zoho,"{'type': 'Point', 'coordinates': [-121.904945,...",-121.904945,37.692934
4,Digg,"{'type': 'Point', 'coordinates': [-122.394523,...",-122.394523,37.764726
...,...,...,...,...
10829,QSGI,"{'type': 'Point', 'coordinates': [-80.041395, ...",-80.041395,26.705331
10830,Oriact,"{'type': 'Point', 'coordinates': [8.4371634, 4...",8.437163,47.088219
10831,AfterLogic,"{'type': 'Point', 'coordinates': [-74.3235539,...",-74.323554,40.793024
10832,EnteGreat Solutions,"{'type': 'Point', 'coordinates': [-86.816068, ...",-86.816068,33.518885


We are now ready to produce the heatmap

In [21]:
# create map of SF Bay Area
sf_map = Map(location=[37.820100, -122.366760],zoom_start=9)

In [22]:
sf_map

In [23]:
# ceate feature group
offices_heatmap = folium.FeatureGroup(name = "Offices")

# adding heatmap to feature group
HeatMap(data=world_heatmap_df[["lat","long"]],radius=15).add_to(offices_heatmap)

# add heatmap and feature group to SF Bay Area map created before
offices_heatmap.add_to(sf_map)

In [24]:
sf_map

Here we can see that, in the Bay Area, most companies are located in the city of SF, followed by Silicon Valley. We will focus on the city of San Francisco

Now we are ready to do geospatial queries

Before that, we will download data from foursquare so that we can compare the locations of different businesses with the locations of offices in our dataset

## Dowload data from foursquare API

We now need to dowload data from foursquare so we can identify locations of places we want to know

We will use .env to send our foursquare tokens safely

In [25]:
load_dotenv()

True

In [26]:
# url to collect data from foursquare

url = 'https://api.foursquare.com/v2/venues/explore'

In [27]:
# foursquare categories to search

nursery_school_c = "4f4533814b9074f6e4fb0107"
preschool_c = "52e81612bcbc57f1066b7a45"
coffee_shop_c = "4bf58dd8d48988d1e0931735"
airport_c = "4bf58dd8d48988d1ed931735"
train_station_c = "4bf58dd8d48988d129951735"
nightlife_spot_c = "4d4b7105d754a06376d81259"
vegetarian_vegan_c = "4bf58dd8d48988d1d3941735"
basketball_court_c = "4bf58dd8d48988d1e1941735"
basketball_stadium_c = "4bf58dd8d48988d18b941735"
salon_barbershop_c = "4bf58dd8d48988d110951735"

As commented at the start, we will use San Francisco in California as a starting point 

In [28]:
# starting searching point is the city of SF, CA
# we use a particular street to make sure we get the correct location

start_loc = "Van Ness Ave, San Francisco, United States of America"
city_data = requests.get(f"https://geocode.xyz/{start_loc}?json=1").json()

In [29]:
print(city_data)

{'success': False, 'error': {'code': '006', 'message': 'Request Throttled. Over Rate limit: up to 2 per sec. See geocode.xyz/pricing', 'requests': '5'}}


In [73]:
san_fran = geocode(start_loc)

In [74]:
san_fran

{'type': 'Point', 'coordinates': [-122.41933, 37.77521]}

In [32]:
san_fran.get('success') == False

True

We can now search for locations in foursquare. We will define a function so we can do this process for every type of location we are searching

We will use the geocode and get_from_dict functions to help in downloading data from foursquare

Since foursquare has a daily call limit, we will prioritise searching for our most valuable places in case we reach the limit

In [33]:
nursery_schools = downl_foursquare(start_loc, nursery_school_c)

In [34]:
nursery_schools[:2]

[{'name': "C5 Children's School",
  'latitud': 37.778864078509734,
  'longitud': -122.41458373129642},
 {'name': 'Judith Baker CD',
  'latitud': 37.776839238148945,
  'longitud': -122.41180216634316}]

In [35]:
preschools = downl_foursquare(start_loc, preschool_c)

In [36]:
preschools[:2]

[{'name': 'LePort School',
  'latitud': 37.776485920736874,
  'longitud': -122.41888653123537},
 {'name': 'Mission Montessori', 'latitud': 37.776597, 'longitud': -122.418816}]

In [37]:
airports = downl_foursquare(start_loc, airport_c)

In [38]:
airports[:2]

[{'name': 'GOGO JETS - San Francisco Private Jet Charter',
  'latitud': 37.79235203792972,
  'longitud': -122.39794403314589},
 {'name': 'Gate 10', 'latitud': 37.76469, 'longitud': -122.423949}]

In [39]:
train_stations = downl_foursquare(start_loc, train_station_c)

In [40]:
train_stations[:2]

[{'name': 'Muni Market & Dolores',
  'latitud': 37.76852533049397,
  'longitud': -122.4251979420112},
 {'name': 'San Francisco Caltrain Station',
  'latitud': 37.77641120337476,
  'longitud': -122.39530950463812}]

In [47]:
nightlife_spots = downl_foursquare(start_loc, nightlife_spot_c)

In [48]:
nightlife_spots[:2]

[{'name': 'Hotel Biron',
  'latitud': 37.77350386431412,
  'longitud': -122.42211040391679},
 {'name': 'Linden Room',
  'latitud': 37.77650338991626,
  'longitud': -122.42279392034109}]

In [49]:
vegan_rests = downl_foursquare(start_loc, vegetarian_vegan_c)

In [50]:
vegan_rests[:2]

[{'name': 'Ananda Fuara',
  'latitud': 37.77769251469199,
  'longitud': -122.41635331222618},
 {'name': 'Shizen',
  'latitud': 37.76819038843062,
  'longitud': -122.42162728988704}]

In [54]:
basketball_stadiums = downl_foursquare(start_loc, basketball_stadium_c)

In [55]:
basketball_stadiums[:2]

[{'name': 'Chase Center', 'latitud': 37.767864, 'longitud': -122.387463},
 {'name': 'Berry Basketball Courts',
  'latitud': 37.771911126215784,
  'longitud': -122.39793109723831}]

In [56]:
salons = downl_foursquare(start_loc, salon_barbershop_c)

In [57]:
salons[:2]

[{'name': 'Hair Play Salon',
  'latitud': 37.77412818312187,
  'longitud': -122.42127120494841},
 {'name': 'Steel + Lacquer',
  'latitud': 37.775773921721424,
  'longitud': -122.41679528915918}]

Once we have the data we need from foursquare, we can start to identify potential office locations

## Filter alternative offices

We will start by filtering offices that are 4km away from our starting point in San Francisco City and number of employees greater than 85. If office is too big, we can rent just the necessary floors

In [75]:
san_fran

{'type': 'Point', 'coordinates': [-122.41933, 37.77521]}

In [76]:
if san_fran.get('success') == False:
        san_fran = {'type': 'Point', 'coordinates': [-122.366760, 37.820100]}

In [77]:
san_fran

{'type': 'Point', 'coordinates': [-122.41933, 37.77521]}

In [78]:
query = {"geojson":
         {"$near": san_fran, 
          "$maxDistance":4000},
         "number_of_employees": {"$gt": 85}}
san_fran_offices = offices.find(query)

In [79]:
sf_city_offices = list(san_fran_offices)

In [80]:
len(sf_city_offices)

79

We can now iterate through the filtered list of SF offices and the lists of venues from foursquare to calculate distances. We will then count the number of cases where the venue is less than 1_500 meters away.

In [81]:
radius = 0.75

In [82]:
near_nursery_schools = count_near_venues(sf_city_offices, nursery_schools, radius)

In [83]:
near_preschools = count_near_venues(sf_city_offices, preschools, radius)

In [84]:
near_airports = count_near_venues(sf_city_offices, airports, radius)

In [85]:
near_train_stations = count_near_venues(sf_city_offices, train_stations, radius)

In [86]:
near_nightlife_spots = count_near_venues(sf_city_offices, nightlife_spots, radius)

In [87]:
near_vegan_rests = count_near_venues(sf_city_offices, vegan_rests, radius)

In [88]:
near_basketball_stadiums = count_near_venues(sf_city_offices, basketball_stadiums, radius)

In [89]:
near_salons = count_near_venues(sf_city_offices, salons, radius)

Since we will be combining the dfs, we need to have a unique name for the 'num_venues' column. We can define a function to change the name

In [90]:
change_column_name(near_nursery_schools, "num_venues", "near_nursery_schools")
near_nursery_schools.head()

,company,near_nursery_schools
0,GitHub 6021ad90292a7a01b909f812,2
1,Clickpass 6021ad90292a7a01b909f5d4,2
2,Tagged 6021ad90292a7a01b909f069,2
3,Yammer 6021ad91292a7a01b90a0176,3
4,Yammer 6021ad91292a7a01b90a0353,3


In [91]:
change_column_name(near_preschools, "num_venues", "near_preschools")
near_preschools.head()

,company,near_preschools
0,GitHub 6021ad90292a7a01b909f812,2
1,Clickpass 6021ad90292a7a01b909f5d4,2
2,Tagged 6021ad90292a7a01b909f069,2
3,Yammer 6021ad91292a7a01b90a0176,3
4,Yammer 6021ad91292a7a01b90a0353,3


In [92]:
change_column_name(near_train_stations, "num_venues", "near_train_stations")
near_train_stations.head()

,company,near_train_stations
0,GitHub 6021ad90292a7a01b909f812,11
1,Clickpass 6021ad90292a7a01b909f5d4,11
2,Tagged 6021ad90292a7a01b909f069,11
3,Yammer 6021ad91292a7a01b90a0176,11
4,Yammer 6021ad91292a7a01b90a0353,11


In [93]:
change_column_name(near_airports, "num_venues", "near_airports")
near_airports.head()

,company,near_airports
0,GitHub 6021ad90292a7a01b909f812,0
1,Clickpass 6021ad90292a7a01b909f5d4,0
2,Tagged 6021ad90292a7a01b909f069,0
3,Yammer 6021ad91292a7a01b90a0176,0
4,Yammer 6021ad91292a7a01b90a0353,0


In [94]:
change_column_name(near_nightlife_spots, "num_venues", "near_nightlife_spots")
near_nightlife_spots.head()

,company,near_nightlife_spots
0,GitHub 6021ad90292a7a01b909f812,27
1,Clickpass 6021ad90292a7a01b909f5d4,27
2,Tagged 6021ad90292a7a01b909f069,28
3,Yammer 6021ad91292a7a01b90a0176,28
4,Yammer 6021ad91292a7a01b90a0353,28


In [95]:
change_column_name(near_vegan_rests, "num_venues", "near_vegan_rests")
near_vegan_rests.head()

,company,near_vegan_rests
0,GitHub 6021ad90292a7a01b909f812,8
1,Clickpass 6021ad90292a7a01b909f5d4,8
2,Tagged 6021ad90292a7a01b909f069,8
3,Yammer 6021ad91292a7a01b90a0176,7
4,Yammer 6021ad91292a7a01b90a0353,7


In [96]:
change_column_name(near_basketball_stadiums, "num_venues", "near_basketball_stadiums")
near_basketball_stadiums.head()

,company,near_basketball_stadiums
0,GitHub 6021ad90292a7a01b909f812,0
1,Clickpass 6021ad90292a7a01b909f5d4,0
2,Tagged 6021ad90292a7a01b909f069,0
3,Yammer 6021ad91292a7a01b90a0176,0
4,Yammer 6021ad91292a7a01b90a0353,0


In [97]:
change_column_name(near_salons, "num_venues", "near_salons")
near_salons.head()

,company,near_salons
0,GitHub 6021ad90292a7a01b909f812,12
1,Clickpass 6021ad90292a7a01b909f5d4,12
2,Tagged 6021ad90292a7a01b909f069,12
3,Yammer 6021ad91292a7a01b90a0176,11
4,Yammer 6021ad91292a7a01b90a0353,11


We can now combine these values and give a total score to each office depending on how well they fit our criteria

In [98]:
near_nursery_schools

,company,near_nursery_schools
0,GitHub 6021ad90292a7a01b909f812,2
1,Clickpass 6021ad90292a7a01b909f5d4,2
2,Tagged 6021ad90292a7a01b909f069,2
3,Yammer 6021ad91292a7a01b90a0176,3
4,Yammer 6021ad91292a7a01b90a0353,3
...,...,...
74,ServePath 6021ad90292a7a01b909f804,0
75,Google 6021ad8f292a7a01b909efc5,0
76,GoGrid 6021ad91292a7a01b909fdf0,0
77,WideOrbit 6021ad91292a7a01b909fc9b,0


In [99]:
comb_df1 = pd.merge(near_nursery_schools, near_preschools, on="company")

In [100]:
comb_df1

,company,near_nursery_schools,near_preschools
0,GitHub 6021ad90292a7a01b909f812,2,2
1,Clickpass 6021ad90292a7a01b909f5d4,2,2
2,Tagged 6021ad90292a7a01b909f069,2,2
3,Yammer 6021ad91292a7a01b90a0176,3,3
4,Yammer 6021ad91292a7a01b90a0353,3,3
...,...,...,...
74,ServePath 6021ad90292a7a01b909f804,0,2
75,Google 6021ad8f292a7a01b909efc5,0,2
76,GoGrid 6021ad91292a7a01b909fdf0,0,1
77,WideOrbit 6021ad91292a7a01b909fc9b,0,1


In [101]:
comb_df2 = pd.merge(comb_df1, near_airports, on="company")

In [102]:
comb_df3 = pd.merge(comb_df2, near_train_stations, on="company")

In [103]:
comb_df4 = pd.merge(comb_df3, near_nightlife_spots, on="company")

In [104]:
comb_df5 = pd.merge(comb_df4, near_vegan_rests, on="company")

In [105]:
comb_df6 = pd.merge(comb_df5, near_basketball_stadiums, on="company")

In [106]:
comb_df7 = pd.merge(comb_df6, near_salons, on="company")

In [107]:
comb_df7

,company,near_nursery_schools,near_preschools,near_airports,near_train_stations,near_nightlife_spots,near_vegan_rests,near_basketball_stadiums,near_salons
0,GitHub 6021ad90292a7a01b909f812,2,2,0,11,27,8,0,12
1,Clickpass 6021ad90292a7a01b909f5d4,2,2,0,11,27,8,0,12
2,Tagged 6021ad90292a7a01b909f069,2,2,0,11,28,8,0,12
3,Yammer 6021ad91292a7a01b90a0176,3,3,0,11,28,7,0,11
4,Yammer 6021ad91292a7a01b90a0353,3,3,0,11,28,7,0,11
...,...,...,...,...,...,...,...,...,...
74,ServePath 6021ad90292a7a01b909f804,0,2,1,1,0,1,1,1
75,Google 6021ad8f292a7a01b909efc5,0,2,1,1,0,1,1,1
76,GoGrid 6021ad91292a7a01b909fdf0,0,1,0,1,0,0,1,1
77,WideOrbit 6021ad91292a7a01b909fc9b,0,1,0,1,0,0,1,1


To calculate the points assigned to each office, we need the values in the df to be ints

In [108]:
comb_df7.columns

Index(['company', 'near_nursery_schools', 'near_preschools', 'near_airports',
       'near_train_stations', 'near_nightlife_spots', 'near_vegan_rests',
       'near_basketball_stadiums', 'near_salons'],
      dtype='object')

In [109]:
df_point_columns = ['near_nursery_schools', 'near_preschools', 'near_airports',
       'near_train_stations', 'near_nightlife_spots', 'near_vegan_rests',
       'near_basketball_stadiums', 'near_salons']

In [110]:
office_scoring = convert_to_int(comb_df7, df_point_columns)

In [111]:
office_scoring.head(10)

,company,near_nursery_schools,near_preschools,near_airports,near_train_stations,near_nightlife_spots,near_vegan_rests,near_basketball_stadiums,near_salons
0,GitHub 6021ad90292a7a01b909f812,2,2,0,11,27,8,0,12
1,Clickpass 6021ad90292a7a01b909f5d4,2,2,0,11,27,8,0,12
2,Tagged 6021ad90292a7a01b909f069,2,2,0,11,28,8,0,12
3,Yammer 6021ad91292a7a01b90a0176,3,3,0,11,28,7,0,11
4,Yammer 6021ad91292a7a01b90a0353,3,3,0,11,28,7,0,11
5,Twitter 6021ad8f292a7a01b909ef8e,3,3,0,12,26,9,0,11
6,Popego 6021ad91292a7a01b90a0323,1,2,0,11,19,8,0,10
7,Popego 6021ad91292a7a01b90a0146,1,2,0,11,19,8,0,10
8,Trulia 6021ad90292a7a01b909f137,0,2,2,1,4,3,0,2
9,Globant 6021ad90292a7a01b909f837,2,4,3,10,8,11,1,7


In [112]:
type(office_scoring['near_nursery_schools'][0])

numpy.int64

We can now give weight to each column depending on how important we think the column is. The weights will be the following, out of a hundred.
- Nursery schools -> 15 (30% have kids, shared with preschools)
- Preschools ------> 15 (30% have kids, shared with nursery schools)
- Airports --------> 15 (account managers travel a lot, shared with train stations)
- Train stations --> 25 (account managers travel a lot, shared with airports)
- Nightlife spots -> 25 (everyone is in party age)
- Vegan rests -----> 2 (only valuable for one member)
- Basketball ------> 2 (only valuable for one member)
- Salon -----------> 1 (only valuable for office dog)

In [113]:
weights_dict = {"near_nursery_schools": 15,
                "near_preschools": 15,
                "near_airports": 15,
                "near_train_stations": 25,
                "near_nightlife_spots": 25,
                "near_vegan_rests": 2,
                "near_basketball_stadiums": 2,
                "near_salons": 1
              }

In [114]:
give_points(office_scoring, weights_dict)

,company,near_nursery_schools,near_preschools,near_airports,near_train_stations,near_nightlife_spots,near_vegan_rests,near_basketball_stadiums,near_salons,near_nursery_schools_points,near_preschools_points,near_airports_points,near_train_stations_points,near_nightlife_spots_points,near_vegan_rests_points,near_basketball_stadiums_points,near_salons_points
0,GitHub 6021ad90292a7a01b909f812,2,2,0,11,27,8,0,12,15.0,15.0,0.0,25.0,25.0,2.0,0.0,1.0
1,Clickpass 6021ad90292a7a01b909f5d4,2,2,0,11,27,8,0,12,15.0,15.0,0.0,25.0,25.0,2.0,0.0,1.0
2,Tagged 6021ad90292a7a01b909f069,2,2,0,11,28,8,0,12,15.0,15.0,0.0,25.0,25.0,2.0,0.0,1.0
3,Yammer 6021ad91292a7a01b90a0176,3,3,0,11,28,7,0,11,15.0,15.0,0.0,25.0,25.0,2.0,0.0,1.0
4,Yammer 6021ad91292a7a01b90a0353,3,3,0,11,28,7,0,11,15.0,15.0,0.0,25.0,25.0,2.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,ServePath 6021ad90292a7a01b909f804,0,2,1,1,0,1,1,1,0.0,15.0,10.5,17.5,0.0,1.4,1.4,0.7
75,Google 6021ad8f292a7a01b909efc5,0,2,1,1,0,1,1,1,0.0,15.0,10.5,17.5,0.0,1.4,1.4,0.7
76,GoGrid 6021ad91292a7a01b909fdf0,0,1,0,1,0,0,1,1,0.0,10.5,0.0,17.5,0.0,0.0,1.4,0.7
77,WideOrbit 6021ad91292a7a01b909fc9b,0,1,0,1,0,0,1,1,0.0,10.5,0.0,17.5,0.0,0.0,1.4,0.7


Finally, we add up all the points!

In [115]:
office_scoring.columns

Index(['company', 'near_nursery_schools', 'near_preschools', 'near_airports',
       'near_train_stations', 'near_nightlife_spots', 'near_vegan_rests',
       'near_basketball_stadiums', 'near_salons',
       'near_nursery_schools_points', 'near_preschools_points',
       'near_airports_points', 'near_train_stations_points',
       'near_nightlife_spots_points', 'near_vegan_rests_points',
       'near_basketball_stadiums_points', 'near_salons_points'],
      dtype='object')

In [116]:
office_scoring["total_points"] = (office_scoring['near_nursery_schools_points'] +
                                  office_scoring['near_preschools_points'] +
                                  office_scoring['near_airports_points'] +
                                  office_scoring['near_train_stations_points'] +
                                  office_scoring['near_nightlife_spots_points'] +
                                  office_scoring['near_vegan_rests_points'] +
                                  office_scoring['near_basketball_stadiums_points'] +
                                  office_scoring['near_salons_points']
                                 )

In [117]:
office_scoring.head()

,company,near_nursery_schools,near_preschools,near_airports,near_train_stations,near_nightlife_spots,near_vegan_rests,near_basketball_stadiums,near_salons,near_nursery_schools_points,near_preschools_points,near_airports_points,near_train_stations_points,near_nightlife_spots_points,near_vegan_rests_points,near_basketball_stadiums_points,near_salons_points,total_points
0,GitHub 6021ad90292a7a01b909f812,2,2,0,11,27,8,0,12,15.0,15.0,0.0,25.0,25.0,2.0,0.0,1.0,83.0
1,Clickpass 6021ad90292a7a01b909f5d4,2,2,0,11,27,8,0,12,15.0,15.0,0.0,25.0,25.0,2.0,0.0,1.0,83.0
2,Tagged 6021ad90292a7a01b909f069,2,2,0,11,28,8,0,12,15.0,15.0,0.0,25.0,25.0,2.0,0.0,1.0,83.0
3,Yammer 6021ad91292a7a01b90a0176,3,3,0,11,28,7,0,11,15.0,15.0,0.0,25.0,25.0,2.0,0.0,1.0,83.0
4,Yammer 6021ad91292a7a01b90a0353,3,3,0,11,28,7,0,11,15.0,15.0,0.0,25.0,25.0,2.0,0.0,1.0,83.0


In [118]:
office_scoring.sort_values(by = 'total_points', ascending = False, inplace = True)

In [119]:
office_scoring.head()

,company,near_nursery_schools,near_preschools,near_airports,near_train_stations,near_nightlife_spots,near_vegan_rests,near_basketball_stadiums,near_salons,near_nursery_schools_points,near_preschools_points,near_airports_points,near_train_stations_points,near_nightlife_spots_points,near_vegan_rests_points,near_basketball_stadiums_points,near_salons_points,total_points
9,Globant 6021ad90292a7a01b909f837,2,4,3,10,8,11,1,7,15.0,15.0,15.0,25.0,25.0,2.0,1.4,1.0,99.4
31,ZEDO 6021ad90292a7a01b909f73b,1,4,5,2,2,13,1,3,10.5,15.0,15.0,25.0,25.0,2.0,1.4,1.0,94.9
11,Huddle 6021ad90292a7a01b909f7a8,1,3,3,10,7,13,1,7,10.5,15.0,15.0,25.0,25.0,2.0,1.4,1.0,94.9
18,Geary Interactive 6021ad90292a7a01b909fa42,1,3,4,7,5,13,1,8,10.5,15.0,15.0,25.0,25.0,2.0,1.4,1.0,94.9
17,Sunrun 6021ad92292a7a01b90a1506,1,3,4,7,5,14,1,8,10.5,15.0,15.0,25.0,25.0,2.0,1.4,1.0,94.9


## Conclusion

The Globant office has received an almost perfect score, so it seems like the ideal candidate

Let's locate the office in a map!

In [120]:
globant = offices.find(
    {"name": "Globant", "offices.city": "San Francisco"},
    {"_id": 0, "name": 1, "geojson.coordinates": 1})

In [121]:
globant = list(globant)
globant

[{'name': 'Globant', 'geojson': {'coordinates': [-122.4041764, 37.7819286]}}]

In [122]:
globant_lat = globant[0].get("geojson").get("coordinates")[1]
globant_long = globant[0].get("geojson").get("coordinates")[0]

In [123]:
globant_lat

37.7819286

In [124]:
globant_long

-122.4041764

In [125]:
# create a map and include a Globant marker

globant_map = folium.Map(location = [globant_lat,globant_long], zoom_start=16)
globant_marker = Marker(location = [globant_lat,globant_long],tooltip="Globant office")
globant_marker.add_to(globant_map)
globant_map

We will now add some markers to locate the venues nearby

For this, we will convert the venue lists we created before into pandas dataframes using a new function

In [126]:
lists_to_be_converted = [nursery_schools,
                         preschools,
                         airports,
                         train_stations,
                         nightlife_spots,
                         vegan_rests,
                         basketball_stadiums,
                         salons
                        ]

In [127]:
resu_dfs = [conv_list_to_df(j) for j in lists_to_be_converted]

In [128]:
for i,row in resu_dfs[0].iterrows():
    nurs_sch = {
        "location": [row["latitud"], row["longitud"]],
        "tooltip": row["name"]
    }
    icon = Icon(color = "green",
                   prefix = "fa",
                   icon = "book",
                   icon_color = "black")
    Marker(**nurs_sch, icon = icon).add_to(globant_map)

In [129]:
for i,row in resu_dfs[1].iterrows():
    prepsch = {
        "location": [row["latitud"], row["longitud"]],
        "tooltip": row["name"]
    }
    icon = Icon(color = "green",
                   prefix = "fa",
                   icon = "book",
                   icon_color = "black")
    Marker(**prepsch, icon = icon).add_to(globant_map)

In [130]:
for i,row in resu_dfs[2].iterrows():
    airp = {
        "location": [row["latitud"], row["longitud"]],
        "tooltip": row["name"]
    }
    icon = Icon(color = "red",
                   prefix = "fa",
                   icon = "plane",
                   icon_color = "black")
    Marker(**airp, icon = icon).add_to(globant_map)

In [131]:
for i,row in resu_dfs[3].iterrows():
    trainst = {
        "location": [row["latitud"], row["longitud"]],
        "tooltip": row["name"]
    }
    icon = Icon(color = "red",
                   prefix = "fa",
                   icon = "train",
                   icon_color = "black")
    Marker(**trainst, icon = icon).add_to(globant_map)

In [132]:
for i,row in resu_dfs[4].iterrows():
    nightl = {
        "location": [row["latitud"], row["longitud"]],
        "tooltip": row["name"]
    }
    icon = Icon(color = "blue",
                   prefix = "fa",
                   icon = "glass",
                   icon_color = "black")
    Marker(**nightl, icon = icon).add_to(globant_map)

In [133]:
for i,row in resu_dfs[5].iterrows():
    vegan = {
        "location": [row["latitud"], row["longitud"]],
        "tooltip": row["name"]
    }
    icon = Icon(color = "orange",
                   prefix = "fa",
                   icon = "spoon",
                   icon_color = "black")
    Marker(**vegan, icon = icon).add_to(globant_map)

In [134]:
for i,row in resu_dfs[6].iterrows():
    prepsch = {
        "location": [row["latitud"], row["longitud"]],
        "tooltip": row["name"]
    }
    icon = Icon(color = "white",
                   prefix = "fa",
                   icon = "soccer-ball-o",
                   icon_color = "black")
    Marker(**prepsch, icon = icon).add_to(globant_map)

In [135]:
for i,row in resu_dfs[7].iterrows():
    prepsch = {
        "location": [row["latitud"], row["longitud"]],
        "tooltip": row["name"]
    }
    icon = Icon(color = "gray",
                   prefix = "fa",
                   icon = "scissors",
                   icon_color = "black")
    Marker(**prepsch, icon = icon).add_to(globant_map)

In [136]:
globant_map